In [ ]:
# 1 ЗАВДАННЯ:

In [1]:
import subprocess
import sys

In [2]:
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2", "surprise", "pybind11>=2.12"])

0

In [3]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic, AlgoBase
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import get_dataset_dir
import os
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
#1 Завантаження датасету ml-100k
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
# 2. Вивід перших 5 рядків завантаженого датасету
raw_ratings = data.raw_ratings
ratings_df = pd.DataFrame(raw_ratings, columns=["user", "item", "rating", "timestamp"])
print(ratings_df.head())

  user item  rating  timestamp
0  196  242     3.0  881250949
1  186  302     3.0  891717742
2   22  377     1.0  878887116
3  244   51     2.0  880606923
4  166  346     1.0  886397596


In [7]:
# 3. Реалізація двох алгоритмів для рекомендаційної системи
# Оберемо алгоритми SVD та KNNBasic
algo1 = SVD()
algo2 = KNNBasic()

In [8]:
# 4. Використання крос-валідації для підбору оптимальних параметрів для обох алгоритмів
# Налаштування параметрів для SVD
param_grid_svd = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01]
}
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=3)
gs_svd.fit(data)

# Найкращі параметри для SVD
print("Best SVD params:", gs_svd.best_params['mae'])

# Налаштування параметрів для KNNBasic
param_grid_knn = {
    'k': [10, 20, 30],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['mae'], cv=3)
gs_knn.fit(data)

# Найкращі параметри для KNN
print("Best KNN params:", gs_knn.best_params['mae'])

Best SVD params: {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.005}
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the 

In [9]:
# 5. Обрання найкращого алгоритму на основі середньої абсолютної помилки (MAE)
# Порівняння MAE
best_algo = gs_svd.best_estimator['mae'] if gs_svd.best_score['mae'] < gs_knn.best_score['mae'] else gs_knn.best_estimator['mae']
if gs_svd.best_score['mae'] < gs_knn.best_score['mae']:
    best_algo = gs_svd.best_estimator['mae']
    print("Best algorithm based on MAE: SVD")
else:
    best_algo = gs_knn.best_estimator['mae']
    print("Best algorithm based on MAE: KNNBasic")

Best algorithm based on MAE: SVD


In [11]:
# 6. Виведення рекомендацій (10 фільмів) для конкретного користувача
# Використовуємо найкращий алгоритм для навчання та передбачення
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Вибираємо конкретного користувача, для прикладу користувача з id '196'
user_id = '182'

# Знаходимо всі фільми, які користувач ще не оцінив
items = set(ratings_df['item'].unique())
rated_items = set(ratings_df[ratings_df['user'] == user_id]['item'])
unrated_items = items - rated_items

# Передбачаємо оцінки для цих фільмів та сортуємо
predictions = [(item, best_algo.predict(user_id, item).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виводимо топ-10 рекомендацій
recommended_items = predictions[:10]
print("Top 10 recommended items for user 182:")
for item, rating in recommended_items:
    print(f"Item: {item}, Predicted rating: {rating:.2f}")

Top 10 recommended items for user 182:
Item: 318, Predicted rating: 4.84
Item: 64, Predicted rating: 4.79
Item: 127, Predicted rating: 4.69
Item: 22, Predicted rating: 4.64
Item: 272, Predicted rating: 4.62
Item: 496, Predicted rating: 4.60
Item: 174, Predicted rating: 4.59
Item: 187, Predicted rating: 4.50
Item: 313, Predicted rating: 4.47
Item: 480, Predicted rating: 4.45


In [ ]:
# Висновки
#У цьому проєкті ми створили рекомендаційну систему на основі датасету MovieLens 100k з використанням бібліотеки Surprise. Було реалізовано два алгоритми — SVD і KNNBasic, а також проведено підбір параметрів за допомогою крос-валідації для покращення якості рекомендацій. На основі середньої абсолютної похибки (MAE) було обрано найкращий алгоритм, за допомогою якого згенеровано 10 рекомендацій для конкретного користувача. У результаті було успішно реалізовано систему персоналізованих рекомендацій, здатну прогнозувати оцінки для фільмів, які користувач ще не переглядав.

In [ ]:
# 2 ЗАВДАННЯ:

In [ ]:
# 1. Завантаження та підготовка даних
# Створимо DataFrame із зразком даних
ratings_dict = {
    'userID': [1, 1, 1, 2, 2, 3, 3, 4, 4, 5],
    'itemID': [1, 2, 3, 1, 4, 2, 3, 4, 5, 1],
    'rating': [5, 3, 4, 4, 5, 3, 5, 4, 2, 1]
}
ratings_df = pd.DataFrame(ratings_dict)

# Використовуємо Surprise Reader для роботи з даними
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userID', 'itemID', 'rating']], reader)

In [ ]:
# 2. Навчання моделі
# Використаємо алгоритм SVD (Single Value Decomposition)
trainset, testset = train_test_split(data, test_size=0.2)
model = SVD()
model.fit(trainset)

In [ ]:
# 3. Оцінка моделі
# Виконуємо передбачення на тестовому наборі даних
predictions = model.test(testset)

# Оцінюємо точність за допомогою метрики RMSE
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 1.5422
RMSE: 1.5421638914048044


In [ ]:
# 4. Отримання рекомендацій для певного користувача
def get_recommendations(model, user_id, item_ids, n=5):
    predictions = [model.predict(user_id, item_id) for item_id in item_ids]
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)
    top_n = sorted_predictions[:n]
    return [(pred.iid, pred.est) for pred in top_n]

# Отримання списку всіх товарів
all_items = ratings_df['itemID'].unique()
# Отримання списку товарів, які користувач ще не оцінив
rated_items_by_user = ratings_df[ratings_df['userID'] == 2]['itemID']
unrated_items = [item for item in all_items if item not in rated_items_by_user.values]

# Отримання рекомендацій
recommendations = get_recommendations(model, user_id=2, item_ids=unrated_items, n=3)
print(f"Recommendations for user 2: {recommendations}")

Recommendations for user 2: [(3, 3.8760364323861864), (5, 3.765938946731381), (2, 3.7659191800437166)]


In [ ]:
# Висновки
# У цьому проекті ми створили просту рекомендаційну систему, яка використовує алгоритм SVD для передбачення рейтингу товарів.
# Ми навчали модель на основі зразка даних про оцінки товарів користувачами, після чого оцінили її точність за допомогою метрики RMSE.
# Значення RMSE дає нам уявлення про те, наскільки добре модель передбачає оцінки користувачів.
# Також ми отримали рекомендації для певного користувача, зокрема для користувача з ID = 2.
# Результати показують, які товари, ймовірно, можуть сподобатися користувачу на основі його вподобань і оцінок інших користувачів.
# Рекомендаційні системи можуть бути дуже корисними для надання персоналізованих пропозицій у різних сферах, таких як онлайн-магазини, стрімінгові платформи, соціальні мережі тощо.